In [5]:
import App
import os
import pickle

In [6]:
UserMaker = App.UserMaker
try:
    with open('G:\\Programming\\major\\data\\users.dat', mode='rb') as file:
        UM = pickle.load(file)
except EOFError as E:
    print(E)
DM = App.DatasetMaker(UM)


In [7]:
from models.User import User, UserData, UserType
from models.Tweet import Tweet
from random import shuffle
id = DM.train_users[0].id

all_train_users = DM.train_users
all_test_users = DM.test_users

shuffle(all_train_users)
shuffle(all_test_users)


In [20]:
x = list(filter(lambda x : x.user_type == UserType.BOT, all_train_users))
y = list(filter(lambda x : x.user_type == UserType.HUMAN, all_train_users))
y_sum = sum([user.num_tweets for user in y])
x_sum = sum([user.num_tweets for user in x])
print(x_sum, y_sum)

396864 1184025


In [4]:
import numpy as np


In [5]:

def get_user_from_pickle(id):
    with open('G:\\Programming\\major\\data\\processed_data\\' + id + '.dat', 'rb') as file:
        U = pickle.load(file)
    return U

U = get_user_from_pickle(DM.train_users[0].id)

In [6]:
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten, Reshape
from keras.layers.convolutional import Conv1D, MaxPooling1D
from keras.utils import np_utils
from keras.layers.recurrent import LSTM
from keras.models import load_model

Using TensorFlow backend.


In [7]:
batch_size=32
num_epochs=20
num_pool=2
num_filters = 128
conv_kernel_width = 3
conv_kernel_height = 200
lstm_output_size= 70
dropout_rate = 0.1
INPUT_UNITS = 32

In [8]:
from keras.layers import Merge

models = []
for i in range(INPUT_UNITS):
    tmodel = Sequential()
    tmodel.add(Conv1D(num_filters, 
                      conv_kernel_width,
                      padding='same',
                      activation='relu',
                      strides=1,
                      input_shape=(20, 200)))
    tmodel.add(MaxPooling1D(pool_size=20))
    models.append(tmodel)

fmodel = Sequential()
fmodel.add(Merge(models, mode='concat'))
fmodel.add(LSTM(64))
fmodel.add(Dropout(0.1))
fmodel.add(Dense(2))
fmodel.add(Activation('sigmoid'))
fmodel.compile(loss='binary_crossentropy',
               optimizer='adam',
               metrics=['accuracy'])

G:\Apps\Conda\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:16: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  app.launch_new_instance()


In [9]:
model = fmodel

In [ ]:
model = Sequential()
model.add(Conv1D(num_filters,
                 conv_kernel_width,
                 padding='same',
                 activation='relu',
                 strides=1,
                 input_shape=(20, 200))) # (20x200) -> (20, 128) 
model.add(MaxPooling1D(pool_size=20)) # (20x128) -> (1, 128)  
model.add(Reshape((128,1)))
model.add(LSTM(batch_size))# 
model.add(Dropout(0.1))
model.add(Dense(64, activation='relu')) # 64
model.add(Dense(2))
model.add(Activation('softmax'))
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [ ]:
for layer in fmodel.layers:
    print(layer,end=',')
    print(' input = ' + str(layer.input_shape) + ' output = ' + str(layer.output_shape))
fmodel.inputs

In [ ]:
class DataGenerator:
    
    def __init__(self, dim_x = 20, dim_y = 200, batch_size=32 , shuffle=True):
        self.dim_x = dim_x
        self.dim_y = dim_y
        self.batch_size = batch_size
        self.user_batch_size = 100
        self.shuffle = shuffle
        
    def __get_exploration_order(self, user_list):
        indexes = np.arange(len(user_list))
        
        if self.shuffle == True:
            shuffle(indexes)
        
        return indexes
    
    def get_tweets_from_users(self, user_list):
        vector_forms = []
        labels = []
        
        for user in user_list:
            U = get_user_from_pickle(user.id)
            for tweet in U.tweets:
                vector_forms.append(tweet.vector_form)
                labels.append(U.user_type.value)
        
        imax = int(len(labels) / self.batch_size)
        for i in range(imax):
            temp_tweets = vector_forms[i*self.batch_size:(i+1) * self.batch_size]
            temp_labels = labels[i*self.batch_size : (i+1) * self.batch_size]
            
            x, y = self.__data_generation(temp_tweets, temp_labels)
            yield x,y
    
    
    def generate(self, user_list):
        while 1:
            exploration_order = self.__get_exploration_order(user_list)
            for i in range(len(user_list)):
                temp_users = [user_list[k] for k in exploration_order[i*self.user_batch_size:
                                                                      (i+1)*self.user_batch_size]]
                for item in self.get_tweets_from_users(temp_users):
                    yield item
                
    def __data_generation(self,  tweets, labels):
        
        X = np.empty((self.batch_size, self.dim_x, self.dim_y))
        Y =  np.empty((self.batch_size), dtype=int)
        
        for i, tweet in enumerate(tweets):
            
            X[i, :, :] = np.array(tweet)
            Y[i] = labels[i]
        
        return X, np_utils.to_categorical(Y, 2)

        

In [10]:
class DataGenerator2: 
    def __init__(self, dim_x = 20, dim_y = 200, batch_size = 1, shuffle=True ):
        self.dim_x = dim_x
        self.dim_y = dim_y
        self.batch_size = batch_size
        self.user_batch_size = 1
        self.shuffle = shuffle
    
    def __get_exploration_order(self, user_list):
        indexes = np.arange(len(user_list))
        
        if self.shuffle == True:
            shuffle(indexes)
        return indexes
    
    def get_user_profile(self, user_list):
        vector_forms = []
        labels = []
        
        for index, user in enumerate(user_list):
            U = get_user_from_pickle(user.id)
            vector_forms.append([tweet.vector_form for tweet in U.tweets])
            labels.append(U.user_type.value)
            # print('list', index, ' ', len(vector_forms[index]), ' ', labels[index])
        
        for i in range(len(labels)):
            
            #Vector forms is like [[user1_tweet1, user1_tweet2, ...], [user2_tweet1, user2_tweet2 ...]]
            #Vector_forms is like [user1_history, user2_history, user3_history, ...]
            #jmax gives us for the ith user, how many times the model can train on their tweets
            
            jmax = int(len(vector_forms[i]) / INPUT_UNITS) 
            
            for j in range(jmax):
                temp_tweets = [vector_forms[i][j * INPUT_UNITS : (j+1) * INPUT_UNITS]]
                # print('l', len(temp_tweets))
                temp_label = [labels[i]]
                x, y = self.__data_generation(temp_tweets, temp_label)
                yield x,y

    
    def generate(self, user_list):
        while 1: 
            exploration_order = self.__get_exploration_order(user_list)
            for i in range(len(user_list)):
                temp_users = [user_list[k] for k in exploration_order[i*self.user_batch_size:
                                                                      (i+1)*self.user_batch_size]]
                for item in self.get_user_profile(temp_users):
                    yield item
                
    def __data_generation(self,  tweets, labels):
        
        X = [np.empty((self.batch_size, self.dim_x, self.dim_y)) for i in range( INPUT_UNITS)]
        Y =  np.empty((self.batch_size), dtype=int)
        

        for i in range(self.batch_size): 
            for k in range(INPUT_UNITS):
                try:
                    X[k][i, :, :] = np.array(tweets[i][k])
                except IndexError as E:
                    print('nexted', i, ' ', k, ' ', tweets[i])
            Y[i] = labels[i]

        return X, np_utils.to_categorical(Y, 2)
    

    

In [11]:
params = {
    'dim_x' : 20,
    'dim_y' : 200,
    'batch_size' : 32,
    'shuffle' : True
}
batch_size = 32
DM.train_tweets = sum([user.num_tweets for user in DM.train_users])
DM.test_tweets = sum([user.num_tweets for user in DM.test_users])
validation_generator = DataGenerator2().generate(DM.test_users)
training_generator = DataGenerator2().generate(DM.train_users)

In [ ]:
params = {
7    'dim_x' : 20,
    'dim_y' : 200,
    'batch_size' : 32,
    'shuffle' : True
}
batch_size = 32
DM.train_tweets = sum([user.num_tweets for user in DM.train_users])
DM.test_tweets = sum([user.num_tweets for user in DM.test_users])
validation_generator = DataGenerator(**params).generate(DM.test_users)
training_generator = DataGenerator(**params).generate(DM.train_users)

In [ ]:
for item in training_generator:
    x,y = item
    print(len(x), y.shape)

In [13]:
model.fit_generator(generator = training_generator,
                    steps_per_epoch = DM.train_tweets/10,
                    validation_data = validation_generator,
                    validation_steps = DM.test_tweets/10,
                    epochs=2)

Epoch 1/1
157424/157424 [==============================] - 10009s - loss: 11.7206 - acc: 0.2688 - val_loss: 12.1384 - val_acc: 0.2428


In [14]:
model.save('G:\\Programming\\major\\trained_model_user_nolstm')
model.save_weights('G:\\Programming\\major\\ut_model_weights_nolstm')

In [ ]:

class DataGenerator:
    
    def __init__(self, dim_x = 20, dim_y = 200, batch_size=32, shuffle=True):
        self.dim_x = dim_x
        self.dim_y = dim_y
        self.batch_size = batch_size
        self.user_batch_size = 100
        self.shuffle = shuffle
        
    def __get_exploration_order(self, user_list):
        indexes = np.arange(len(user_list))
        
        if self.shuffle == True:
            shuffle(indexes)
        
        return indexes
    
    def get_tweets_from_users(self, user_list):
        vector_forms = []
        labels = []
        
        for user in user_list:
            U = get_user_from_pickle(user.id)
            for tweet in U.tweets:
                vector_forms.append(tweet.vector_form)
                labels.append(U.user_type.value)
        
        imax = int(len(labels) / self.batch_size)
        for i in range(imax):
            temp_tweets = vector_forms[i*self.batch_size:(i+1) * self.batch_size]
            temp_labels = labels[i*self.batch_size : (i+1) * self.batch_size]
            
            x, y = self.__data_generation(temp_tweets, temp_labels)
            yield x,y
    
    def generate(self, user_list):
        while 1:
            exploration_order = self.__get_exploration_order(user_list)
            for i in range(len(user_list)):
                temp_users = [user_list[k] for k in exploration_order[i*self.user_batch_size:
                                                                      (i+1)*self.user_batch_size]]
                for item in self.get_tweets_from_users(temp_users):
                    yield item
                
    def __data_generation(self,  tweets, labels):
        
        X = np.empty((self.batch_size, self.dim_x, self.dim_y))
        Y =  np.empty((self.batch_size), dtype=int)
        
        for i, tweet in enumerate(tweets):
            
            X[i, :, :] = np.array(tweet)
            Y[i] = labels[i]
        
        return X, np_utils.to_categorical(Y, 2)

        

In [ ]:
from keras.layers.merge import Concatenate
from keras.layers import Reshape
n_model = []
for i in range(batch_size):
    n_model.append(Sequential())
    n_model[i].add(Conv1D(num_filters,
                     conv_kernel_width,
                     padding='same',
                     activation='relu',
                     strides=1,
                     input_shape=(20,200,)))
    n_model[i].add(MaxPooling1D(pool_size=20))

dim1 = 1
dim2 = 20 / num_pool
dim3 = num_filters


final_model = Sequential()
final_model.add(Merge([m for m in n_model], mode='concat'))
print(final_model.output_shape) # input_shape=[(20,200) for i in range(batch_size)]))#, input_shape=batch_size*num_filters)
final_model.add(LSTM(lstm_output_size))
final_model.add(Dropout(0.1))
final_model.add(Dense(64, activation='relu'))
final_model.add(Dense(2))
final_model.add(Activation('softmax'))
final_model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])


In [ ]:
final_model.fit_generator(generator = training_generator,
                    steps_per_epoch = DM.train_tweets//batch_size,
                    validation_data = validation_generator,
                    validation_steps = DM.test_tweets//batch_size,
                    epochs=10)

In [ ]:
import matplotlib.pyplot as plt


In [ ]:
plt.plot([i for i in range(1,11)], [89.9, 90.46, 91.47, 89.84, 89.87, 73.54, 89.82, 91.5, 91.03, 91.15])
plt.xlabel("Number of Epochs")
plt.ylabel("Accuracy in %")
plt.title("Model Accuracy ")
plt.axis([0,11,50,100])
plt.xticks([i for i in range(1, 11)])
plt.show()

In [ ]:
model = load_model("G:\\Programming\\major\\trained_model")

In [ ]:
from vectorize import vectorize
from vectorize import preprocess

In [ ]:
tweet = "This is a good thing"
vector_form = vectorize.vectorize_filter(
                preprocess.lemmatize_filter(
                    preprocess.tokenize_filter(tweet)))

In [ ]:
import numpy as np
model.predict(np.reshape(np.array(vector_form), (1,20,200)))